In [29]:
import json
import os
import re
import pandas as pd

In [30]:
constraints = ['delta2_no_users_attacker_in_conv', 'delta2_no_users']
for constraint in constraints:
    all_cnts = 0
    user_cnts = 0
    with open('/scratch/wiki_dumps/expr_with_matching/%s/data/all.json'%constraint) as f:
        for line in f:
            conv_id, clss, conversation = json.loads(line)
            if 'user' in conversation['action_feature'][0]['page_title'].lower():
                user_cnts += 1
            all_cnts += 1
    print(constraint, user_cnts / all_cnts)

delta2_no_users_attacker_in_conv 0.6727976766698935
delta2_no_users 0.46580928481806777


In [18]:
def get_activity_cnt(ns, year):

    users = set([])
    os.system('tar -xzf /scratch/wiki_dumps/wikipedia_talk_corpus/comments_%s_%d.tar.gz' % (ns, year))
    data_dir = "comments_%s_%d" % (ns, year)
    activity_dict = {}
    for month in range(1, 13):
        activity_dict[month] = []
    total = []
    pages = set([])
    for _, _, filenames in os.walk(data_dir):
        for filename in filenames:
            if re.match("chunk_\d*.tsv", filename):
                df = pd.read_csv(os.path.join(data_dir, filename), sep = "\t")
                pages = pages | set(df['page_id'].values.tolist())
    os.system('rm -r comments_%s_%d' % (ns, year))
    return pages

In [21]:
pages = set([])
for year in range(2001, 2016):
    for ns in ['article', 'user']:
        pages = pages | get_activity_cnt(ns, year)

In [22]:
print(len(pages))

13740651


In [31]:
with open("/scratch/wiki_dumps/expr_with_matching/user_features/updated.json") as f:
    inp = json.load(f)
user_features = {}
for conv, users in inp:
    user_features[conv] = users

In [32]:
def attacker_is_flyby(conversation, user_features):
    actions = conversation['action_feature']
    end_time = max([action['timestamp_in_sec'] for action in actions])
    for action in actions:
        if action['timestamp_in_sec'] == end_time:
            if not('user_text' in action):
                return True
            user = action['user_text']
            if user_features[user]['edits_on_this_talk_page'] == 0:
                return True
            else: return False

In [33]:
constraints = 'delta2_no_users'
bads = []
all_data = {}
tests = []
flybys = 0
all_bads = 0
with open('/scratch/wiki_dumps/expr_with_matching/%s/data/test.json'%constraint) as f:
    for line in f:
        conv_id, clss, conversation = json.loads(line)
        tests.append(conv_id)

In [34]:
with open('/scratch/wiki_dumps/expr_with_matching/%s/data/all.json'%constraint) as f:
    for line in f:
        conv_id, clss, conversation = json.loads(line)
        if attacker_is_flyby(conversation, user_features[conv_id]):
            all_data[conv_id] = [conv_id, clss, conversation]
            if not(clss):
                bads.append([conv_id, clss, conversation])
                flybys += 1
        if not(clss):
            all_bads += 1
print('Number of real flyby attackers:', flybys / all_bads)

Number of real flyby attackers: 0.8300920117105813


In [37]:
left = []
with open('/scratch/wiki_dumps/expr_with_matching/delta2_only_flybys/data/train.json', 'w') as w:
    for bad in bads:
        matched_id = bad[2]['action_feature'][0]['good_conversation_id']
        if matched_id in all_data:
            if not(bad in tests):
                w.write(json.dumps(bad) + '\n')
                w.write(json.dumps(all_data[matched_id]) + '\n')
            else:
                left.append(bad)
                left.append(all_data[matched_id])

with open('/scratch/wiki_dumps/expr_with_matching/delta2_only_flybys/data/test.json', 'w') as w:
    for data in left:
        w.write(json.dumps(data) + '\n')